# 从引用关系文件中获取共同引用关系

In [1]:
import json
import pandas as pd
import itertools    

### Dataframe分组统计，按cited_id分组

In [3]:
path2 = 'allPapers_citation_id2id_dict.json'
with open(path2, 'r', encoding='utf-8') as fr_5:
    citation_id2id_dict = json.load(fr_5)

In [4]:
def groupby_citedID_func(index2citation_dict):
    # 分组
    citation_id2id_data = pd.DataFrame(index2citation_dict.values())
    groupby_citedID_data = citation_id2id_data.groupby(['cited_id'])
    groupby_list = list(groupby_citedID_data)

    groupby_citedID_dict = {}
    # 遍历列表中的所有分组数据
    for i in range(len(groupby_list)):
        data_i = groupby_list[i]
        citedID_i = data_i[0]
        citingID_list_i = [e for e in data_i[1]['citing_id']]
        groupby_citedID_dict[citedID_i] = citingID_list_i
        
    return groupby_citedID_dict

In [48]:
def groupby_to_cocitation_func(groupby_citedID_dict):
    cocitingTuple_list = []

    # 遍历统计每两个citing_id之间的共引文献数
    for key, value in groupby_citedID_dict.items():
        if len(value) >= 2:
            combinations_i = [e for e in itertools.combinations(value,2)]
            # combinations_list_i = [e for e in combinations_i]
            cocitingTuple_list.extend(combinations_i)
  
    return cocitingTuple_list

In [49]:
def cociting_data2count_func(cocitingTuple_list):
    df = pd.DataFrame(cocitingTuple_list)
    # 统计重复行次数（即有共引关系的两个施引文献对的共引次数）
    df_count = df.groupby(by=[0, 1], as_index=False).size()
    df_count = df_count.rename(columns={0: 'CitingID_1', 1: 'CitingID_2', 'size':'Weight'})
    df_count_sorted = df_count.sort_values(by='CitingID_1', ascending=True)
    
    return df_count_sorted

In [6]:
# 获取共引关系数据
groupby_citedID_dict = groupby_citedID_func(citation_id2id_dict)
cocitingTuple_list = groupby_to_cocitation_func(groupby_citedID_dict)
cocitingData_sorted = cociting_data2count_func(cocitingTuple_list)

filePath = 'allPapers_cociting_data.csv'
with open(filePath, 'w', newline='', encoding='utf-8') as fw:
    cocitingData_sorted.to_csv(fw, index=False)

### Dataframe分组统计，按citing_id分组

In [2]:
def groupby_citingID_func(index2citation_dict):
    # 将上述字典信息转换为dataframe格式
    citation_id2id_data = pd.DataFrame(index2citation_dict.values())
    groupby_citingID_data = citation_id2id_data.groupby(['citing_id'])
    groupby_list = list(groupby_citingID_data)

    groupby_citingID_dict = {}
    # 遍历列表中的所有分组数据
    for i in range(len(groupby_list)):
        data_i = groupby_list[i]
        citingID_i = data_i[0]
        citedID_list_i = [e for e in data_i[1]['cited_id']]
        groupby_citingID_dict[citingID_i] = citedID_list_i
        
    return groupby_citingID_dict

In [3]:
def groupby_to_cocitation_func(citingID_groupby_dict):
    citingID_list = [e for e in citingID_groupby_dict.keys()]
    
    cocitingData_list = []
    # 遍历统计每两个citing_id之间的共引文献数
    for i in range(len(citingID_list)):
        citingID_i = citingID_list[i]
        citedID_list_i = citingID_groupby_dict[citingID_i]
        ## 引用论文ID_j
        for j in range(i+1, len(citingID_list)):
            citingID_j = citingID_list[j]
            citedID_list_j = citingID_groupby_dict[citingID_j]
            # 统计引用论文ID_i和引用论文ID_j的共同引用论文数
            same_citedID_ij = list(set(citedID_list_i) & set(citedID_list_j))
            same_count_ij = len(same_citedID_ij)
            temp = {}
            temp['CitingID_1'] = citingID_i
            temp['CitingID_2'] = citingID_j
            temp['Weight'] = same_count_ij
            cocitingData_list.append(temp)
  
    return cocitingData_list

In [5]:
# 将所有引用关系转换为共同引用关系数据
groupby_citingID_dict = groupby_citingID_func(citation_id2id_dict)
cocitingData_list = groupby_to_cocitation_func(groupby_citingID_dict)
cocitingData_df = pd.DataFrame(cocitingData_list)
cocitingData_df_sorted = cocitingData_df.sort_values(by='CitingID_1', ascending=True)

filePath = './allPapers_cociting_data.csv'
with open(filePath, 'w', newline='', encoding='utf-8') as fw:
    cocitingData_df_sorted.to_csv(fw, index=False)